In [1]:
module PSO 

export init_population,optimize
type Particle{T,N}
    x::Array{T,N} # position
    x_cost::Float64
    v::Array{T,N} # velocity
    xh::Array{T,N} # personal best
    xh_cost::Float64
    xg::Array{T,N} # global best
    nb::Vector{Int}
end

const c1 = 1e-2
const c2 = 1e-2

function init_population(x0,N)
    [Particle(x0,1e6,randn(2),
    x0,1e6,
    x0,[1:N;]) for i=1:N];
end
    
function velocity_update(p::Particle)
    for d in eachindex(p.v)
        @inbounds p.v[d] += c1*rand()*(p.xh[d]-p.x[d]) + c2*rand()*(p.xg[d]-p.x[d])
    end
end
function position_update(p::Particle)
    for d in eachindex(p.x)
        @inbounds p.x[d] += p.v[d]
    end
end 
function evaluate(f,g)
    #     Threads.@threads 
    for p in g
        p.x_cost = f(p.x)
    end
end

function optimize(f,g; T=1000)
    for t = 1:T
        evaluate(f,g)
        
        for p in g
            if p.x_cost < p.xh_cost
                p.xh = copy(p.x)
                p.xh_cost = p.x_cost
            end

            idx = indmin(g[i].x_cost for i in p.nb)
            p.xg = g[p.nb[idx]].x

            velocity_update(p)
            position_update(p)
        end
    end
    y0,index = findmin(p.xh_cost for p in g)
    x0 = g[index].xh
    x0,y0
end

end # end of module

PSO

In [17]:
using PSO
f(x) = norm(x)
group = init_population(rand(2),10);
optimize(f,group;T=1000)

([0.241385,-0.00556958],0.24144943061559335)